In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 182 kB 59.4 MB/s 
     |████████████████████████████████| 7.6 MB 43.7 MB/s 


Reddit dataset classification using transformer

Importing the libraries.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import torch
import torch.nn as nn
from transformers.pipelines import TextClassificationPipeline
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


1st function: selecting 10 words from each comments. The comments without atleast 10 words have eos by default.

In [ ]:
def tenwords(str):
  tokens=str.split()[:10]
  str1=''

  for i in range(len(tokens)):
    if i==0:
      str1+=tokens[i]
    else:

      str1=str1+" "+tokens[i]
  return str1

2nd function: as part of the preprossesing I am removing cleaning punctuation.

In [ ]:
def cleaningPunc(str):
  str=re.sub(r'[^\w\s]','',str)
  return str

33d function: as part of the preprossesing I am removing the urls.

In [ ]:
def urlremove(sentence):
  sentence=re.sub(r'http\S+', '', sentence)
  sentence=re.sub(r'www\S+', '', sentence)
  return sentence



  #sentence=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',sentence)
  #sentence = re.sub('@[^\s]+','',sentence)
  #setence = re.sub('http[^\s]+','',sentence)
  #sentence=re.sub('https:\/\/.*','',sentence)

  #sentence=re.sub(r'^https?:\/\/.*[\r\n]*','',sentence)


4th function: as part preprocessing removing stopwords like[I,is....]

In [ ]:
def stopwordsRemove(str):
  str1=""
  stop_words=stopwords.words('english')
  str=str.split()

  for word in str:
    if not word in stop_words:
      str1+=word+' '
  return str1



5th function: as part of the preprossesing removing digits like 1,2...

In [ ]:
def cleaningdigits(str):
  str=re.sub(r'[\d+]','',str)
  return str

6th function: stemming but according to the vocabulary .

In [ ]:
def lemmatization(str):
  lemma=WordNetLemmatizer()
  str1=''
  tokens=str.split()
  for word in tokens:
    store=lemma.lemmatize(word)
    str1+=store+' '
  return str1



In [ ]:
#reading the csv file
dataFrame=pd.read_csv("reddit_data.csv")
#dataFrame.head()


#removing the columns that is not necessary now.
dataFrame.drop('parent_id',inplace=True,axis=1)
dataFrame.drop('length',inplace=True,axis=1)
dataFrame.drop('size_range',inplace=True,axis=1)


#dataFrame.head()

#removing duplicates from the dataset
dataFrame=dataFrame.drop_duplicates()

#removing float values(Nan) from the dataset
dataFrame=dataFrame[dataFrame['text'].notnull()]
dataFrame=dataFrame[dataFrame['topic'].notnull()]
#dataFrame['text'] = dataFrame['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
#spliting the dataset
train,test=train_test_split(dataFrame,test_size=0.3)

#separating the x_train data and y_train data. Converting them into numpy array.
x_train=train['text'].to_numpy()
y_train=train['topic'].to_numpy()
x_test=test['text'].to_numpy()
y_test=test['topic'].to_numpy()

x_test=x_test[:10]
y_test=y_test[:10]
x_train=x_train[:10]
y_train=y_train[:10]

#preprocessing [taking 10 words,cleaning punctuation, stopwords Remove,cleaningdightis, lemmatization ]
count=0
for i in range(len(x_train)):
  sentence=x_train[i]

  sentence=tenwords(sentence)

  sentence=cleaningPunc(sentence)

  sentence=stopwordsRemove(sentence)

  sentence=cleaningdigits(sentence)

  sentence=lemmatization(sentence)

  sentence=urlremove(sentence)
  x_train[i]=sentence



for i in range(len(x_test)):
  sentence=x_test[i]

  sentence=tenwords(sentence)

  sentence=cleaningPunc(sentence)

  sentence=stopwordsRemove(sentence)

  sentence=cleaningdigits(sentence)

  sentence=lemmatization(sentence)

  sentence=urlremove(sentence)
  x_test[i]=sentence






In [ ]:
#one hot encoding
token_index={}
counter=0
for sentence in x_train:
  tokens=sentence.split()
  for token in tokens:
    if not token in token_index:
      token_index.update({token:counter+1})
      counter=counter+1

#print(token_index)


In [ ]:
#checking vocabulary size and indices
#unique_words=token_index.keys()
#nique_values=token_index.values()

In [ ]:
# Set max_length to 10 representaing 10 words
max_length = 10
# Create a tensor of dimension 3 named results whose every elements are initialized to 0
results  = np.zeros(shape = (len(x_train),
                            max_length,
                            max(token_index.values()) + 1))
print(results.shape)

results_test  = np.zeros(shape = (len(x_test),
                            max_length,
                            max(token_index.values()) + 1))


print(results_test.shape)

(10, 10, 53)
(10, 10, 53)


In [ ]:
# One-hot vector corresponding to the word
# Iterate over enumerate(samples) enumerate object
for i, documents in enumerate(x_train):

# Convert enumerate object to list and iterate over resultant list
  for j, considered_word in list(enumerate(documents.split())):

    # set the value of index variable equal to the value of considered_word in token_index
    index = token_index.get(considered_word)


    # In the previous zero tensor: results, set the value of elements with their positional index as [i, j, index] = 1.
    results[i, j, index] = 1.




In [ ]:
# One-hot vector corresponding to the word
# Iterate over enumerate(samples) enumerate object
for i, documents in enumerate(x_test):

# Convert enumerate object to list and iterate over resultant list
  for j, considered_word in list(enumerate(documents.split())):

    # set the value of index variable equal to the value of considered_word in token_index
    index = token_index.get(considered_word)


    # In the previous zero tensor: results, set the value of elements with their positional index as [i, j, index] = 1.
    results_test[i, j, index] = 1.




Implementing text classification on reddit data

In [ ]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"),
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
#vocab_size = 20000  # Only consider the top 20k words
#maxlen = 200  # Only consider the first 200 words of each movie review

#(x_train, y_train), (x_val, y_val) = imdb.load_data(num_words=vocab_size)
#print(len(x_train), "Training sequences")
#print(len(x_val), "Validation sequences")
y_train_dict={}
counter=0
for i,label in enumerate(y_train):
  if not label in y_train_dict:
      y_train_dict.update({label:counter+1})
      counter=counter+1
  y_train[i]=y_train_dict[label]

for i,label in enumerate(y_test):
  if not label in y_train_dict:
    y_train_dict.update({label:counter+1})
    counter=counter+1
  y_test[i]=y_train_dict[label]


In [ ]:
maxlen=10
vocab_size=len(token_index)
#x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
#y_train= tf.keras.preprocessing.sequence.pad_sequences(y_train, maxlen=maxlen)

#x_val = tf.keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 1  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(2, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
x_train=np.asarray(results).astype(np.int)

y_train=np.asarray(y_train).astype(np.int)
x_test=np.asarray(results_test).astype(np.int)

y_test=np.asarray(y_test).astype(np.int)
history = model.fit(x_train, y_train,
                    batch_size=128, epochs=2,
                    validation_data=(results_test, y_test)
                   )

Epoch 1/2


<ipython-input-128-a509144ac480>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_train=np.asarray(results).astype(np.int)
<ipython-input-128-a509144ac480>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1

ValueError: ignored

In [ ]:
results = model.evaluate(x_test, y_test, verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))


ValueError: ignored

**Implementing contextiual word and then mlp**



Bert***


In [ ]:
MLP = Sequential()
MLP.add(InputLayer(input_shape=(720, ))) # input layer not complete
MLP.add(Dense(256, activation='relu')) # hidden layer 1
MLP.add(Dense(256, activation='relu')) # hidden layer 2
MLP.add(Dense(10, activation='softmax')) # output layer

# summary
MLP.summary()

SyntaxError: ignored

In [ ]:
MLP.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])
MLP.fit(train, train, epochs=20, batch_size=128)

NameError: ignored

In [ ]:

# evaluate performance
test_loss, test_acc = MLP.evaluate(train, test,batch_size=128,verbose=0)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

index=np.argmax(MLP.predict(digit, verbose=0))

NameError: ignored

**Transformer implementation from scratch**

In [ ]:



class SelfAttention(nn.Module):
    def __init__(self,embed_size,heads):
        super(SelfAttention,self).__init__()
        self.embed_size=embed_size
        self.heads=heads
        self.head_dim=embed_size//heads

        assert (self.head_dim*heads==embed_size),"Embed size needs to be div by heads"

        self.values=nn.Linear(self.head_dim,self.head_dim,bias=False)
        self.keys=nn.Linear(self.head_dim,self.head_dim,bias=False)
        self.queries=nn.Linear(self.head_dim,self.head_dim,bias=False)

        self.fc_out=nn.Linear(heads*self.head_dim,embed_size)

    def forward(self,values,keys,query,mask):
        N=query.shape[0]
        value_len,key_len,query_len=values.shape[1],keys.shape[1],query.shape[1]

        values=values.reshape(N,value_len,self.heads,self.head_dim)

        keys=keys.reshape(N,key_len,self.head,self.head_dim)

        queries=query.reshape(N,query_len,self.heads,self.head_dim)
        values=self.values(values)
        key=self.keys(keys)
        queries-self.queries(queries)
        energy=torch.einsum("nqhd,nkhd->nhqk",[queries,keys])

        if mask is not None:
            energy=energy.masked_fill(mask==0,float("-1e20"))
        attention=torch.softmax(energy/(self.embed_size**(1/2)),dim=3)
        out=torch.einsum("nhql,nlhd->nqhd",[attention,values]).reshape(N,query_len,self.heads*self.head_dim)

        out=self.fc_out(out)
        return out

class TransformerBlock(nn.Module):
    def __init__(self,embed_size,heads,dropout,forward_expansion):
        super(TransformerBlock,self).__init__()
        self.attention=SelfAttention(embed_size,heads)
        self.norm1=nn.LayerNorm(embed_size)
        self.norm2=nn.LayerNorm(embed_size)

        self.feed_forward=nn.Sequential(nn.Linear(embed_size,forward_expansion*embed_size),nn.Relu(),nn.Linear(forward_expansion*embed_size,embed_size))
        self.dropout=nn.Dropout(dropout)
    def forward(self,value,key,query,mask):
        attention=self.attention(value,key,query,mask)
        x=self.dropout(self.norm1(attention+query))
        forward=self.feed_forward(x)
        out=self.dropout(self.norm2(forward+x))
        return out

class Encoder(nn.Module):
    def __init(self,src_vocab_size,embed_size,num_layers,heads,device,forward_expansion,dropout,max_length):
        super(Encoder,self).__init__()
        self.embed_size=embed_size
        self.device=device
        self.word_embedding=nn.Embedding(src_vocab_size,embed_size)
        self.postional_embedding=nn.Embedding(max_length,embed_size)
        self.layers=nn.ModuleList([TransformerBlock(embed_size,heads,dropout=dropout,forward_expansion=forward_expansion)] for _ in range(num_layers))
        self.dropout=nn.Dropout(dropout)

    def forward(self,x,mask):
        N,seq_length=x.shape
        positions=torch.arrange(0,seq_length).expand(N,seq_length).to(self.device)
        out=self.dropout(self.word_embedding(x)+self.postional_embedding(positions))
        for layer in self.layers:
            out=layer(out,out,out,mask)
        return out

class DecoderBlock(nn.Module):
    def __init__(self,embed_size,heads,forward_expansion,dropout,device):
        super(DecoderBlock,self).__init__()
        self.attention=SelfAttention(embed_size,heads)
        self.norm=nn.LayerNorm(embed_size)
        self.transformer_block=TransformerBlock(embed_size,heads,dropout,forward_expansion)
        self.dropout=nn.Dropout(dropout)
    def forward(self,x,value,key,src_mask,trg_mask):
        attention=self.attention(x,x,x,trg_mask)
        query=self.dropout(self.norm(attention+x))
        out=self.transformer_block(value,key,query,src_mask)

class Decoder(nn.Module):
    def __init__(self,trg_vocab_size,embed_size,num_layers,heads,forward_expansion,dropout,device,max_length):
        super(Decoder,self).__init__()
        self.device=device
        self.word_embedding=nn.Embedding(trg_vocab_size,embed_size)
        self.layers=nn.ModuleList([DecoderBlock(embed_size,heads,forward_expansion,dropout,device)
        for _ in range(num_layers)])
        self.fc_out=nn.Linear(embed_size,trg_vocab_size)
        self.dropout=nn.Dropout(dropout)

    def forward(self,enc_out,src_mask,trg_mask):
        N,seq_length= x.shape
        positions=torch.arrange(0,seq_length).expand(N,seq_length).to(self.device)
        x=self.dropout(self.word_embedding(x)+self.postion_embedding(positions))
        for layer in self.layers:
            x=layer(x,enc_out,enc_out,src_mask,trg_mask)
        out=self.fc_out(x)
        return out
class Transformer(nn.Module):
    def __init__(self,src_vocab_size,trg_vocab_size,src_pad_idx,trg_pad_idx,embed_size=256,num_layers=6,forward_expansion=4,heads=8,dropout=8,device='colab',max_length=10):
        super (Transformer,self).__init__()
        self.encoder=Encoder(src_vocab_size,embed_size,num_layers,heads,device,forward_expansion,dropout,max_length)
        self.decoder=Decoder(trg_vocab_size,embed_size,num_layers,heads,forward_expansion,dropout,device,max_length)
        self.src_pad_idx=src_pad_idx
        self.trg_pad_idx=trg_pad_idx
    def make_src_mask(self,src):
        src_mask=(src!=self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)
    def make_trg_mask(self,trg):
        N,trg_len=trg.shape
        trg_maks=torch.tril(torch.one((trg_len,trg_len))).expand(N,1,trg_len,trg_len)
        return trg_mask.to(self.device)
    def forward(self,src,trg):
        src_mask=self.make_src_mask(src)
        trg_mask=self.make_trg_mask(trg)
        enc_src=self.encoder(src,src_mask)
        out=self.decoder(trg.enc_sr,src_mask,trg_mask)
        return out










NameError: ignored

In [ ]:
def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P